## Exemplo de uso Python OpenAI

### **Importante**

Este notebook utiliza um recurso que só existe nas versões da lib openai anteriores à 1.x. Para executá-lo, instale a openai==0.28.1. O outro notebook permite usar a versão mais recente da lib.

In [1]:
import os
import openai  # pip install openai
from dotenv import load_dotenv  # pip install python-dotenv
from configparser import ConfigParser, ExtendedInterpolation

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config.ini', 'UTF-8')

load_dotenv()

os.environ['REQUESTS_CA_BUNDLE'] = './petrobras-ca-root.pem'

openai.api_type = 'azure'
openai.api_base = config['OPENAI']['OPENAI_API_BASE']
openai.api_version = config['OPENAI']['OPENAI_API_VERSION']
openai.api_key = 'not-used'

# Configuração do prefixo do APIM, que é diferente do prefixo da Azure (não documentada)
from openai.api_resources.abstract import APIResource
APIResource.azure_api_prefix = config['OPENAI']['AZURE_OPENAI_PREFIX']

# Configura headers
headers = {
    'api-key': config['OPENAI']['OPENAI_API_KEY'],
    'Content-Type': 'application/json',
    'Cache-Control': 'no-cache'
}

In [2]:
def get_completion(engine, headers, prompt, **kwargs):

    try:
        return openai.Completion.create(
            # Contact your team admin to get the name of your engine or model deployment.  
            # This is the name that they used when they created the model deployment.
            prompt=prompt,
            engine=engine,
            headers=headers,
            **kwargs
        )

    except openai.error.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")

    except openai.error.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")

    except openai.error.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")

    except openai.error.InvalidRequestError as e:
        # Handle connection error here
        print(f"Invalid Request Error: {e}")

    except openai.error.RateLimitError as e:
        # Handle rate limit error
        print(f"OpenAI API request exceeded rate limit: {e}")

    except openai.error.ServiceUnavailableError as e:
        # Handle Service Unavailable error
        print(f"Service Unavailable: {e}")

    except openai.error.Timeout as e:
        # Handle request timeout
        print(f"Request timed out: {e}")


def get_embedding(text, engine, headers, **kwargs):

    try:
        response = openai.Embedding.create(
            # Contact your team admin to get the name of your engine or model deployment.  
            # This is the name that they used when they created the model deployment.
            input=text,
            engine=engine,
            headers=headers,
            **kwargs
        )
    
        embeddings = response['data'][0]['embedding']
        return embeddings

    except openai.error.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")

    except openai.error.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")

    except openai.error.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")

    except openai.error.InvalidRequestError as e:
        # Handle connection error here
        print(f"Invalid Request Error: {e}")

    except openai.error.RateLimitError as e:
        # Handle rate limit error
        print(f"OpenAI API request exceeded rate limit: {e}")

    except openai.error.ServiceUnavailableError as e:
        # Handle Service Unavailable error
        print(f"Service Unavailable: {e}")

    except openai.error.Timeout as e:
        # Handle request timeout
        print(f"Request timed out: {e}")


# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, engine, headers, max_response_tokens=500):
    
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
        headers = headers
    )
    return response['choices'][0]['message']['content']


# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

### Exemplo Chat API

In [3]:
base_system_message = "Você é um assistente que conhece tudo sobre podcasts. Você é capaz de responder perguntas sobre o podcast, sobre os episódios e sobre os convidados."
system_message = f"{base_system_message.strip()}"

user_message = "A área de tecnologia da Petrobras, chamada de TIC está criando um programa de podcast através do qual pretende levar aos colaboradores da empresa temas sobre tecnologia tais como ChatGPT e IA generativa, por exemplo. Alguma sugestão de nome para esse podcast? Indique por favor uma lista de dez nomes bacanas para esse podcast."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, engine=config['OPENAI']['CHATGPT_MODEL'], headers=headers, max_response_tokens=max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
Você é um assistente que conhece tudo sobre podcasts. Você é capaz de responder perguntas sobre o podcast, sobre os episódios e sobre os convidados.

[USER]
A área de tecnologia da Petrobras, chamada de TIC está criando um programa de podcast através do qual pretende levar aos colaboradores da empresa temas sobre tecnologia tais como ChatGPT e IA generativa, por exemplo. Alguma sugestão de nome para esse podcast? Indique por favor uma lista de dez nomes bacanas para esse podcast.

[ASSISTANT]
1. "TechPetro: Conexão e Inovação"
2. "TIC Talk: Diálogos Tecnológicos da Petrobras"
3. "PetroTech: A Nova Era Digital"
4. "Onda Tecnológica Petrobras"
5. "Petro IA: Transformando o Futuro"
6. "PetroDigital: A Revolução TIC"
7. "Conexão Petrobras: Tecnologia e Inovação"
8. "TIC para Todos: A Tecnologia na Petrobras"
9. "PetroChat: Conversas sobre IA e Tecnologia"
10. "PetroInova: O Podcast de Tecnologia da Petrobras"




### Exemplo Embeddings API

In [ ]:
get_embedding(text="Olá, mundo!", engine=config['OPENAI']['EMBEDDINGS_MODEL'], headers=headers)

### Exemplo Completion API (deprecated)

A API de Completion foi descontinuada pela Open AI.

In [ ]:
#get_completion(engine=config['OPENAI']['COMPLETIONS_MODEL'], headers=headers, prompt="Era uma vez, um menino chamado Romeu e uma menina chamada Julieta")